<a href="https://colab.research.google.com/github/ba18406/Lab001/blob/master/Lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Load the data from ``jester-data-1.csv''
#The data is from http://eigentaste.berkeley.edu/dataset/ and it contains the ratings of 101 jokes from 24,983 users
#You can find the jokes in the website http://eigentaste.berkeley.edu/dataset/jester_dataset_1_joke_texts.zip
#Check the dataset description to understand why we replace random cells as 99!
#Using the helper function provided in the helper_function notebook (or create your own), label 10% of the dataset cells as 99. This is your validation set. Keep the the actual values of the cells so you can use them later.
#Use latent factor modeling to infer the hidden ratings of the users (they are labeled as "99" in the dataset) on the training set
#Calculate the performance (e.g., MSE) of the algorithm on the validation dataset
#Repeat the two points above changing hyper-parameters (i.e., learning rate, number of iterations of SVD, number of latent factors, etc.) as needed to get good results (you can create multiple validation sets if you want, and run a bootstrap!)
#Once you're happy, make predictions for the test dataset
#(If you have time) Use pandas to find the best- and the worst-rated jokes

In [0]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", delimiter=',', header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  rated = np.where(arr!=99)
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [4]:
arr = df.values
validationData, idx = replace(arr)
print(validationData)

[[ 74.    99.     8.79 ...  99.    99.    99.  ]
 [100.     4.08  -0.29 ...  99.    99.     1.07]
 [ 49.    99.    99.   ...  99.    99.    99.  ]
 ...
 [ 39.    99.    99.   ...  99.    99.    99.  ]
 [ 37.    99.    99.   ...  99.    99.    99.  ]
 [ 72.     2.43   2.67 ...  99.    99.    99.  ]]


In [5]:
n_latent_factors = 5
user_ratings = df.values

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

print(latent_user_preferences.shape)
print(latent_item_features.shape)

(24983, 5)
(101, 5)


In [0]:

def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.00001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [0]:
sgd()

1177.7838427321885


In [8]:
user_ratings = validationData

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[1.47594032, 2.05103828, 1.33789428, ..., 1.03420511, 1.32732281,
        1.6328081 ],
       [1.48905616, 2.29038504, 1.50828475, ..., 1.12249952, 2.29698313,
        1.44116262],
       [1.50755008, 2.33918399, 1.40681629, ..., 1.17037176, 2.15059429,
        1.5196365 ],
       ...,
       [1.49884583, 1.76797317, 1.32635154, ..., 1.04549223, 1.7760686 ,
        1.3034059 ],
       [1.11723385, 1.76218006, 1.0587623 , ..., 0.70871943, 1.41389068,
        1.19485515],
       [1.74640444, 2.77082251, 1.61876265, ..., 1.32792421, 2.35965535,
        1.84088499]])

In [10]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 1.4759403164223108)","(99.0, 2.05103827605503)","(8.79, 1.3378942770192093)","(-9.66, 1.6170172822289965)","(-8.16, 0.894799385370858)","(-7.52, 1.4478561977857258)","(-8.5, 1.0000577555618937)","(-9.85, 1.2004537814735805)","(4.17, 1.9165837670394064)","(-8.98, 1.5181953575800837)","(-4.76, 2.089401269878051)","(-8.5, 1.3255165769365531)","(99.0, 2.06099115814586)","(-7.18, 1.057769639104716)","(8.45, 1.7053720494338815)","(-7.18, 1.96742250574351)","(-7.52, 0.4845447784519789)","(-7.43, 1.1353934613355825)","(-9.81, 1.44883844291572)","(-9.85, 1.6184619929018282)","(-9.85, 0.8024465333185334)","(-9.37, 1.4775621240169603)","(1.5, 1.9630336894752274)","(-4.37, 1.0260448757461995)","(99.0, 2.1919603646106016)","(-8.5, 1.213777308370349)","(1.12, 1.691819342339613)","(99.0, 1.567376287986002)","(2.86, 1.5021102375661008)","(99.0, 1.2519168831392524)","(-7.43, 1.2781493391688985)","(2.14, 0.9443876771390937)","(-4.08, 1.321888322308453)","(-9.08, 1.367647808261226)","(7.82, 0.9009731322344708)","(5.05, 0.5663198466226675)","(4.95, 1.5881351568058424)","(-9.17, 0.9924037695834328)","(-8.4, 0.9627368187656493)","(-8.4, 1.5780375603270358)",...,"(8.59, 1.245202462343972)","(3.59, 0.6824588700983636)","(-6.84, 1.101296607698133)","(-9.03, 1.237279393705654)","(2.82, 0.6224035829292929)","(-1.36, 1.2353635397545306)","(-9.08, 0.689826333682556)","(8.3, 1.2700249071514862)","(5.68, 0.8658953141669604)","(99.0, 1.3203484926573967)","(99.0, 1.959304829827992)","(99.0, 1.2274526500758476)","(99.0, 1.5239077201907163)","(99.0, 2.0872217272695024)","(99.0, 1.4307268712705403)","(99.0, 0.7927441385134502)","(99.0, 1.159371875332208)","(-9.42, 1.9099251092123404)","(99.0, 1.1244018543942624)","(99.0, 1.478727048174029)","(99.0, 1.40048488533549)","(-7.72, 1.9453267023753427)","(99.0, 1.405063792595119)","(99.0, 0.49019879923282145)","(99.0, 1.3153358448948214)","(99.0, 1.3343535804840767)","(99.0, 0.673366024594751)","(99.0, 0.8362356651458315)","(99.0, 1.5412585238296306)","(99.0, 1.7544905528681185)","(2.82, 0.6803915546261642)","(99.0, 2.0835470878560454)","(99.0, 1.1010200995816328)","(99.0, 1.4799418249598815)","(99.0, 1.6194694351918362)","(99.0, 1.0646035807226348)","(-5.63, 1.2722270174939267)","(99.0, 1.0342051067933002)","(99.0, 1.327322809900901)","(99.0, 1.6328080990458498)"
1,"(100.0, 1.4890561610633133)","(4.08, 2.290385040988144)","(-0.29, 1.5082847503254537)","(6.36, 1.0383594240322709)","(4.37, 0.7398587858144754)","(-2.38, 2.2560118044653352)","(-9.66, 1.0379196326039135)","(-0.73, 1.8579940875529606)","(-5.34, 1.7768593469377223)","(8.88, 1.336058580560563)","(9.22, 1.8752815155848754)","(6.75, 1.2394561280213656)","(8.64, 1.9511510220945687)","(4.42, 0.7739023575262995)","(99.0, 2.0572430589208826)","(4.56, 1.7679838288626386)","(-0.97, 0.8840116918956847)","(4.66, 1.3641166563160247)","(-0.68, 1.3435834338579624)","(3.3, 1.9476684791625647)","(-1.21, 1.4056982855629054)","(0.87, 1.7132655540172592)","(8.64, 2.03496495465427)","(8.35, 1.2228036836637919)","(9.17, 2.30134830752031)","(0.05, 0.9452868323419661)","(7.57, 2.0174337238327578)","(4.71, 1.2923487726226006)","(0.87, 1.2763856682933978)","(-0.39, 1.5964503501821208)","(6.99, 1.4975244525561169)","(6.5, 0.7366967457403308)","(99.0, 1.5608299141985147)","(7.14, 1.704668472718342)","(9.03, 1.2673927661797577)","(-1.8, 0.9046097753915792)","(0.73, 1.4354974055193066)","(7.09, 1.6318897189261423)","(3.4, 1.1130373302837049)","(-0.87, 1.585520339224931)",...,"(99.0, 1.9972625326124187)","(-3.35, 1.081296250326555)","(-9.03, 1.257550101359844)","(4.47, 1.3848207218005861)","(4.08, 1.2096801254129448)","(-3.83, 1.826401883833475)","(8.74, 0.760574541195549)","(1.12, 1.4330533752381378)","(0.78, 1.4604911725064569)","(7.52, 1.03505

In [0]:
#Ex1

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
df = pd.read_csv('jester-data-1.csv')
print(df)

In [0]:
arr = df.values
print(arr.shape)

In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

In [0]:
new_arr[idx[0][0], idx[1][0]]

In [0]:
#Ex2

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
user_ratings_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_ratings.csv")
user_features_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_features.csv")
item_features_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/item_features.csv")

In [0]:
user_features_df["key"] = 0
user_features_df["user_id"] = range(user_features_df.shape[0])
item_features_df["key"] = 0
item_features_df["item_id"] = range(item_features_df.shape[0])

merged_df = pd.merge(user_features_df, item_features_df, left_index=True, on="key")
merged_df.drop(labels='key', axis=1, inplace=True)  # drop the "key" column

In [0]:
merged_df[["item_id", "user_id"]]

In [0]:
merged_df["rating"] = map(lambda ids: user_ratings_df.values[ids[1]][ids[2]], 
                          merged_df[["user_id", "item_id"]].itertuples())

train = merged_df.dropna()
test = merged_df[merged_df.isnull().any(axis=1)]

In [0]:
n_latent_factors = 2

user_ratings = user_ratings_df.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

user_features = user_features_df.values
item_features = item_features_df.values

user_features = np.concatenate([np.ones(shape=(user_features.shape[0],1)), user_features], axis=1)
item_features = np.concatenate([np.ones(shape=(item_features.shape[0],1)), item_features], axis=1)

user_features_weights = np.random.random((user_ratings.shape[0], user_features.shape[1]))
item_features_weights = np.random.random((user_ratings.shape[1], item_features.shape[1]))

print(user_features)

In [0]:
def predict_rating(user_id, item_id):
    """
    Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    
    user_score = user_features_weights[user_id].dot(user_features[user_id])
    item_score = item_features_weights[item_id].dot(item_features[item_id])
    #print(user_preference.dot(item_preference), user_score, item_score)
    return user_preference.dot(item_preference) + user_score + item_score


def train(user_id, item_id, rating, alpha=0.001, 
          latent_feature_weight_decay=0.1,
          user_weight_decay=0.01, item_weight_decay=0.0001):
    #print(item_id)
    prediction_rating = predict_rating(user_id, item_id)
    err = prediction_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id] + latent_feature_weight_decay * latent_user_preferences[user_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values + latent_feature_weight_decay * latent_item_features[item_id])
    
    user_features_weights[user_id] -=alpha * err * (user_features[user_id] + user_weight_decay * user_features_weights[user_id])
    item_features_weights[item_id] -=alpha * err * (item_features_weights[item_id] + item_weight_decay * item_features_weights[item_id])
    
    return err


def sgd(iterations=30000):
    """ 
    Iterate over all users and all items and train for 
    a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
    mse = (np.array(error) ** 2).mean()          
    print(mse)

In [0]:
for _ in range(10): 
    sgd()  # Note decreasing values with increasing iterations

In [0]:
predictions = np.zeros(shape=(latent_user_preferences.shape[0], latent_item_features.shape[0]))
print(user_features_weights)
print(item_features_weights)
for user_id in range(latent_user_preferences.shape[0]):
    for item_id in range(latent_item_features.shape[0]):
        predictions[user_id, item_id] =  predict_rating(user_id, item_id)

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = user_ratings_df.columns

In [0]:
#Ex3

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
Image(filename='books.png')

In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_ratings.csv")

In [0]:
n_latent_factors = 2

user_ratings = data.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
latent_item_features

In [0]:
latent_user_preferences

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [0]:
sgd()  # Note how the MSE decreases with the number of iterations

In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns

In [0]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction